In [ ]:
! pip install 'mordred[full]'
! pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=2b322212e17b3365c79ebc4e7c78e542fe8583dc25f9fe50b6a1340def0a89eb
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 

In [ ]:
import pandas as pd
import numpy as np

from mordred import Calculator, descriptors

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [ ]:
import pandas as pd
import numpy as np

hcv_drugs = {
    'Name' : ['Dasabuvir', 'Sofosbuvir', 'Daclatasvir', 'Velpatasvir'],
    'Smiles' : ['COc1c(-c2ccc3cc(NS(C)(=O)=O)ccc3c2)cc(-n2ccc(=O)[nH]c2=O)cc1C(C)(C)C',
                'CC(C)OC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@@H](n2ccc(=O)[nH]c2=O)[C@](C)(F)[C@@H]1O)Oc1ccccc1',
                'COC(=O)N[C@H](C(=O)N1CCC[C@H]1c1ncc(-c2ccc(-c3ccc(-c4cnc([C@@H]5CCCN5C(=O)[C@@H](NC(=O)OC)C(C)C)[nH]4)cc3)cc2)[nH]1)C(C)C',
                'COC[C@H]1C[C@@H](c2nc(-c3ccc4c(c3)COc3cc5c(ccc6nc([C@@H]7CC[C@H](C)N7C(=O)[C@@H](NC(=O)OC)C(C)C)[nH]c65)cc3-4)c[nH]2)N(C(=O)[C@H](NC(=O)OC)c2ccccc2)C1'],
    'ChEMBL ID' : ['CHEMBL3137312', 'CHEMBL1259059', 'CHEMBL2023898', 'CHEMBL3545062'],
    'pIC50 (Kumar et. al., web-app)' : [5.49938, 6.29241, 6.02152, 5.40850],
    'Drug documentation' : ['Inhibited a panel of genotype 1a and 1b NS5B polymerases with median IC50 values of 2.8 nM (range 2.4 nM to 4.2 nM; n = 3) and 3.7 nM (range 2.2 nM to 10.7 nM; n = 4), respectively',
                     'Inhibited the polymerase activity of the recombinant NS5B from HCV genotype 1b, 2a, 3a and 4a with IC50 values ranging from 0.7 to 2.6 microM',
                     'It is a substrate of CYP3A. In vitro, daclatasvir did not inhibit (IC50 greater than 40 microM) CYP enzymes 1A2, 2B6, 2C8, 2C9, 2C19, or 2D6.',
                     'It is an inhibitor of the HCV NS5A protein, which is required for viral replication. It is also an inhibitor of BCRP, OATP1B1, and OATP1B3 with IC50 values of 0.30, 1.5, and 0.26 microM, respectively.'],
    'Conversion to pIC50 of the documentation' : ['8.5528 M median at genotype 1a NS5B polymerase and 8.4318 M median at 1b NS5B polymerase',
                                              'Range from 6.1549 to 5.5850 M',
                                              '4.3979 M, did not inhibit any of the CYP enzymes in the documentation',
                                              '6.5229 M at BCRP, 5.8239 M at OATP1B1, and 6.5850 M at OATP1B3'],
    'Documentation source' : ['https://www.accessdata.fda.gov/drugsatfda_docs/label/2015/206619s003lbl.pdf',
                              'https://www.accessdata.fda.gov/drugsatfda_docs/label/2013/204671s000lbl.pdf',
                              'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=9803a6ff-8a3e-4c64-b3d0-7825c7123bf2',
                              'https://www.accessdata.fda.gov/drugsatfda_docs/label/2016/208341s000lbl.pdf ; https://doi.org/10.1016/B978-0-12-820018-6.00011-9']
}

hcv_drugs_df = pd.DataFrame(hcv_drugs)
hcv_drugs_df

,Name,Smiles,ChEMBL ID,"pIC50 (Kumar et. al., web-app)",Drug documentation,Conversion to pIC50 of the documentation,Documentation source
0,Dasabuvir,COc1c(-c2ccc3cc(NS(C)(=O)=O)ccc3c2)cc(-n2ccc(=...,CHEMBL3137312,5.49938,Inhibited a panel of genotype 1a and 1b NS5B p...,8.5528 M median at genotype 1a NS5B polymerase...,https://www.accessdata.fda.gov/drugsatfda_docs...
1,Sofosbuvir,CC(C)OC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@@H](n...,CHEMBL1259059,6.29241,Inhibited the polymerase activity of the recom...,Range from 6.1549 to 5.5850 M,https://www.accessdata.fda.gov/drugsatfda_docs...
2,Daclatasvir,COC(=O)N[C@H](C(=O)N1CCC[C@H]1c1ncc(-c2ccc(-c3...,CHEMBL2023898,6.02152,"It is a substrate of CYP3A. In vitro, daclatas...","4.3979 M, did not inhibit any of the CYP enzym...",https://dailymed.nlm.nih.gov/dailymed/drugInfo...
3,Velpatasvir,COC[C@H]1C[C@@H](c2nc(-c3ccc4c(c3)COc3cc5c(ccc...,CHEMBL3545062,5.40850,"It is an inhibitor of the HCV NS5A protein, wh...","6.5229 M at BCRP, 5.8239 M at OATP1B1, and 6.5...",https://www.accessdata.fda.gov/drugsatfda_docs...


# **Mordred descriptors**

In [ ]:
# Define a function to calculate all Mordred descriptors

def AllMordredDescriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]
    df = calc.pandas(mols)
    return df

In [ ]:
hcv_df_descriptors = AllMordredDescriptors(hcv_drugs_df['Smiles'])

100%|██████████| 4/4 [00:04<00:00,  1.01s/it]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
hcv_df_descriptors

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,27.738099,20.956737,0,0,42.865435,2.505661,5.011321,42.865435,1.224727,4.489674,...,10.647732,71.976532,493.167142,7.954309,3716,60,192.0,226.0,13.597222,7.291667
1,27.827563,22.925756,0,0,43.592387,2.529940,4.920453,43.592387,1.210900,4.497017,...,10.533349,87.239339,529.162544,8.140962,4161,58,188.0,219.0,14.736111,7.736111
2,42.068697,31.249861,0,0,69.891647,2.463529,4.828198,69.891647,1.294290,4.916224,...,10.876896,110.326732,738.385331,7.099859,15525,88,286.0,341.0,18.222222,11.944444
3,51.656602,35.386971,0,0,85.882277,2.561421,5.078253,85.882277,1.321266,5.120900,...,11.244091,123.007241,882.406461,7.415180,23175,117,360.0,439.0,20.138889,14.166667


# **RDKit MACCS keys**

In [ ]:
# Define a function to calculate the MACCS keys

def MACCSFingerprints(data):
    maccs_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  AllChem.GetMACCSKeysFingerprint(mol)
        mfpts = np.array(fpts)
        maccs_fpts.append(mfpts)
    return np.array(maccs_fpts)

In [ ]:
hcv_df_maccs = pd.DataFrame(MACCSFingerprints(hcv_drugs_df['Smiles']))

In [ ]:
hcv_df_maccs

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
# Reset all of the indexes

hcv_drugs_df.reset_index(drop=True, inplace=True)
hcv_df_descriptors.reset_index(drop=True, inplace=True)
hcv_df_maccs.reset_index(drop=True, inplace=True)

In [ ]:
final_df = pd.concat([hcv_drugs_df, hcv_df_descriptors, hcv_df_maccs], axis=1)

In [ ]:
final_df

,Name,Smiles,ChEMBL ID,"pIC50 (Kumar et. al., web-app)",Drug documentation,Conversion to pIC50 of the documentation,Documentation source,ABC,ABCGG,nAcid,...,157,158,159,160,161,162,163,164,165,166
0,Dasabuvir,COc1c(-c2ccc3cc(NS(C)(=O)=O)ccc3c2)cc(-n2ccc(=...,CHEMBL3137312,5.49938,Inhibited a panel of genotype 1a and 1b NS5B p...,8.5528 M median at genotype 1a NS5B polymerase...,https://www.accessdata.fda.gov/drugsatfda_docs...,27.738099,20.956737,0,...,1,1,1,1,1,1,1,1,1,0
1,Sofosbuvir,CC(C)OC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@@H](n...,CHEMBL1259059,6.29241,Inhibited the polymerase activity of the recom...,Range from 6.1549 to 5.5850 M,https://www.accessdata.fda.gov/drugsatfda_docs...,27.827563,22.925756,0,...,1,1,1,1,1,1,1,1,1,0
2,Daclatasvir,COC(=O)N[C@H](C(=O)N1CCC[C@H]1c1ncc(-c2ccc(-c3...,CHEMBL2023898,6.02152,"It is a substrate of CYP3A. In vitro, daclatas...","4.3979 M, did not inhibit any of the CYP enzym...",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,42.068697,31.249861,0,...,1,1,1,1,1,1,1,1,1,0
3,Velpatasvir,COC[C@H]1C[C@@H](c2nc(-c3ccc4c(c3)COc3cc5c(ccc...,CHEMBL3545062,5.40850,"It is an inhibitor of the HCV NS5A protein, wh...","6.5229 M at BCRP, 5.8239 M at OATP1B1, and 6.5...",https://www.accessdata.fda.gov/drugsatfda_docs...,51.656602,35.386971,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
selected_keys = ['Name', 52, 62, 65, 69, 79, 94, 109, 117, 122, 124, 148]

In [ ]:
# final_df = final_df.applymap(lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, object) else x)

In [ ]:
final_df

,Name,Smiles,ChEMBL ID,"pIC50 (Kumar et. al., web-app)",Drug documentation,Conversion to pIC50 of the documentation,Documentation source,ABC,ABCGG,nAcid,...,157,158,159,160,161,162,163,164,165,166
0,Dasabuvir,COc1c(-c2ccc3cc(NS(C)(=O)=O)ccc3c2)cc(-n2ccc(=...,CHEMBL3137312,5.49938,Inhibited a panel of genotype 1a and 1b NS5B p...,8.5528 M median at genotype 1a NS5B polymerase...,https://www.accessdata.fda.gov/drugsatfda_docs...,27.738099,20.956737,0,...,1,1,1,1,1,1,1,1,1,0
1,Sofosbuvir,CC(C)OC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@@H](n...,CHEMBL1259059,6.29241,Inhibited the polymerase activity of the recom...,Range from 6.1549 to 5.5850 M,https://www.accessdata.fda.gov/drugsatfda_docs...,27.827563,22.925756,0,...,1,1,1,1,1,1,1,1,1,0
2,Daclatasvir,COC(=O)N[C@H](C(=O)N1CCC[C@H]1c1ncc(-c2ccc(-c3...,CHEMBL2023898,6.02152,"It is a substrate of CYP3A. In vitro, daclatas...","4.3979 M, did not inhibit any of the CYP enzym...",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,42.068697,31.249861,0,...,1,1,1,1,1,1,1,1,1,0
3,Velpatasvir,COC[C@H]1C[C@@H](c2nc(-c3ccc4c(c3)COc3cc5c(ccc...,CHEMBL3545062,5.40850,"It is an inhibitor of the HCV NS5A protein, wh...","6.5229 M at BCRP, 5.8239 M at OATP1B1, and 6.5...",https://www.accessdata.fda.gov/drugsatfda_docs...,51.656602,35.386971,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
selected_keys_df = final_df[selected_keys]

In [ ]:
selected_keys_df

,Name,52,62,65,69,79,94,109,117,122,124,148
0,Dasabuvir,0,1,1,1,0,1,0,1,1,1,1
1,Sofosbuvir,0,1,1,1,0,1,1,1,1,1,1
2,Daclatasvir,0,1,1,0,1,0,0,1,1,0,1
3,Velpatasvir,0,1,1,0,1,0,1,1,1,0,1


In [ ]:
selected_keys_df.columns = [str(col) for col in selected_keys_df.columns]

### **Cosine similarity method**

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

n = len(selected_keys_df)
similarity_matrix = pd.DataFrame(np.zeros((n, n)), columns=selected_keys_df['Name'], index=selected_keys_df['Name'])

for i in range(n):
    for j in range(n):
        vec1 = selected_keys_df.iloc[i, 1:].values
        vec2 = selected_keys_df.iloc[j, 1:].values
        similarity_matrix.iloc[i, j] = cosine_similarity(vec1, vec2)

print(similarity_matrix)

Name         Dasabuvir  Sofosbuvir  Daclatasvir  Velpatasvir
Name                                                        
Dasabuvir     1.000000    0.942809     0.721688     0.668153
Sofosbuvir    0.942809    1.000000     0.680414     0.755929
Daclatasvir   0.721688    0.680414     1.000000     0.925820
Velpatasvir   0.668153    0.755929     0.925820     1.000000


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cmpds_of_int = pd.read_csv('/content/drive/MyDrive/Thesis Files (2023)/compounds_of_interest.csv')
cmpds_of_int

,Molecule.ChEMBL.ID,Smiles,Standard.Relation,Standard.Value,Document.ChEMBL.ID,Molar,pIC50,Bioactivity.Class,ATSC1v,AATSC1dv,...,62,65,69,79,94,109,117,122,124,148
0,CHEMBL1765744,CC[C@@H]1CO[C@H](C(=O)NC2CCCCC2)CN1c1cc(-c2ccc...,'>',10000.00,CHEMBL1765070,1.000000e-05,5.000000,intermediate,88.658452,1.402261,...,1,1,1,1,1,1,1,1,1,1
1,CHEMBL1765731,Nc1nc(-c2ccc3c(N)n[nH]c3c2)cc(N2CCO[C@@H](C(=O...,'>',10000.00,CHEMBL1765070,1.000000e-05,5.000000,intermediate,173.893236,0.911050,...,1,1,1,1,1,1,1,1,1,1
2,CHEMBL1765743,CNc1nc(-c2ccc3c(N)n[nH]c3c2)cc(N2C[C@@H](C(=O)...,'>',10000.00,CHEMBL1765070,1.000000e-05,5.000000,intermediate,89.152043,1.395133,...,1,1,1,1,1,1,1,1,1,1
3,CHEMBL1765742,CC[C@@H]1CO[C@H](C(=O)Nc2ccccc2)CN1c1cc(-c2ccc...,'=',3162.28,CHEMBL1765070,3.162280e-06,5.500000,active,93.958537,1.265163,...,1,1,1,1,1,1,1,1,1,1
4,CHEMBL3666720,COc1cc2nc(-c3cccc(NC(=O)CN4CCOCC4)c3)nc(Nc3ccc...,'>',3000.00,CHEMBL3639077,3.000000e-06,5.522879,active,-30.860302,1.154308,...,1,1,1,1,1,1,1,1,1,1
5,CHEMBL4103214,Cc1n[nH]cc1-c1cc2nc(CN3CCC[C@@H]3CO)[nH]c(=O)c...,'=',1200.00,CHEMBL4033719,1.200000e-06,5.920819,active,-46.711828,1.301327,...,1,1,1,1,1,1,1,1,1,1
6,CHEMBL1614774,Nc1nc(-c2ccc3c(N)[nH]nc3c2)cc(N2CCO[C@H](C(=O)...,'=',794.33,CHEMBL1765070,7.943300e-07,6.099999,active,173.893236,0.911050,...,1,1,1,1,1,1,1,1,1,1
7,CHEMBL1765741,C[C@@H]1CO[C@H](C(=O)Nc2ccccc2)CN1c1cc(-c2ccc3...,'=',158.49,CHEMBL1765070,1.584900e-07,6.799998,active,171.814364,1.078380,...,1,1,1,1,1,1,1,1,1,1
8,CHEMBL3956456,COc1cc(C(=O)NC(CN2CCOCC2)c2ccccc2)ccc1-c1cn[nH]c1,'=',107.00,CHEMBL3886981,1.070000e-07,6.970616,active,-6.672503,0.846074,...,1,1,1,1,1,1,1,1,1,1
9,CHEMBL1922143,COc1cccc(CNC(=O)c2[nH]c3nc(-c4cn[nH]c4)ccc3c2C...,'=',91.00,CHEMBL1921670,9.100000e-08,7.040959,active,12.560462,1.015873,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
cmpds_of_int = cmpds_of_int.loc[[0, 9, 13, 15, 17]]
cmpds_of_int

,Molecule.ChEMBL.ID,Smiles,Standard.Relation,Standard.Value,Document.ChEMBL.ID,Molar,pIC50,Bioactivity.Class,ATSC1v,AATSC1dv,...,62,65,69,79,94,109,117,122,124,148
0,CHEMBL1765744,CC[C@@H]1CO[C@H](C(=O)NC2CCCCC2)CN1c1cc(-c2ccc...,'>',10000.0,CHEMBL1765070,1.000000e-05,5.000000,intermediate,88.658452,1.402261,...,1,1,1,1,1,1,1,1,1,1
9,CHEMBL1922143,COc1cccc(CNC(=O)c2[nH]c3nc(-c4cn[nH]c4)ccc3c2C...,'=',91.0,CHEMBL1921670,9.100000e-08,7.040959,active,12.560462,1.015873,...,1,1,1,1,1,1,1,1,1,1
13,CHEMBL1215171,COc1ccc2c(c1)C[C@H](C(=O)Nc1ccc(-c3cn[nH]c3)cc...,'=',48.0,CHEMBL1212924,4.800000e-08,7.318759,active,-9.979652,1.141927,...,1,1,1,1,1,1,1,1,1,1
15,CHEMBL1215524,CN(C)CCOc1cc(-c2cn[nH]c2)c(F)cc1NC(=O)[C@H]1Cc...,'=',33.0,CHEMBL1212924,3.300000e-08,7.481486,active,36.918406,1.082118,...,1,1,1,1,1,1,1,1,1,1
17,CHEMBL1215304,COc1ccc2c(c1)C[C@H](C(=O)Nc1ccc(-c3cn[nH]c3)cc...,'=',10.0,CHEMBL1212924,1.000000e-08,8.000000,active,51.819228,1.036981,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
selected_keys_cmpds_of_int = ['Molecule.ChEMBL.ID', '52', '62', '65', '69', '79', '94', '109', '117', '122', '124', '148']

In [ ]:
cmpds_int_keys_only = cmpds_of_int[selected_keys_cmpds_of_int]
cmpds_int_keys_only

,Molecule.ChEMBL.ID,52,62,65,69,79,94,109,117,122,124,148
0,CHEMBL1765744,1,1,1,1,1,1,1,1,1,1,1
9,CHEMBL1922143,1,1,1,1,1,1,1,1,1,1,1
13,CHEMBL1215171,1,1,1,1,1,1,1,1,1,1,1
15,CHEMBL1215524,1,1,1,1,1,1,1,1,1,1,1
17,CHEMBL1215304,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
cmpds_int_keys_only = cmpds_int_keys_only.rename(columns={'Molecule.ChEMBL.ID': 'Name'})

In [ ]:
cmpds_int_keys_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    5 non-null      object
 1   52      5 non-null      int64 
 2   62      5 non-null      int64 
 3   65      5 non-null      int64 
 4   69      5 non-null      int64 
 5   79      5 non-null      int64 
 6   94      5 non-null      int64 
 7   109     5 non-null      int64 
 8   117     5 non-null      int64 
 9   122     5 non-null      int64 
 10  124     5 non-null      int64 
 11  148     5 non-null      int64 
dtypes: int64(11), object(1)
memory usage: 608.0+ bytes


In [ ]:
selected_keys_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    4 non-null      object
 1   52      4 non-null      int64 
 2   62      4 non-null      int64 
 3   65      4 non-null      int64 
 4   69      4 non-null      int64 
 5   79      4 non-null      int64 
 6   94      4 non-null      int64 
 7   109     4 non-null      int64 
 8   117     4 non-null      int64 
 9   122     4 non-null      int64 
 10  124     4 non-null      int64 
 11  148     4 non-null      int64 
dtypes: int64(11), object(1)
memory usage: 512.0+ bytes


In [ ]:
cmpds_int_keys_only.reset_index(drop=True, inplace=True)
selected_keys_df.reset_index(drop=True, inplace=True)

In [ ]:
cosine_similarity_df = pd.concat([cmpds_int_keys_only, selected_keys_df], axis=0)
cosine_similarity_df

,Name,52,62,65,69,79,94,109,117,122,124,148
0,CHEMBL1765744,1,1,1,1,1,1,1,1,1,1,1
1,CHEMBL1922143,1,1,1,1,1,1,1,1,1,1,1
2,CHEMBL1215171,1,1,1,1,1,1,1,1,1,1,1
3,CHEMBL1215524,1,1,1,1,1,1,1,1,1,1,1
4,CHEMBL1215304,1,1,1,1,1,1,1,1,1,1,1
0,Dasabuvir,0,1,1,1,0,1,0,1,1,1,1
1,Sofosbuvir,0,1,1,1,0,1,1,1,1,1,1
2,Daclatasvir,0,1,1,0,1,0,0,1,1,0,1
3,Velpatasvir,0,1,1,0,1,0,1,1,1,0,1


In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

n = len(cosine_similarity_df)
similarity_matrix = pd.DataFrame(np.zeros((n, n)), columns=cosine_similarity_df['Name'], index=cosine_similarity_df['Name'])

for i in range(n):
    for j in range(n):
        vec1 = cosine_similarity_df.iloc[i, 1:].values
        vec2 = cosine_similarity_df.iloc[j, 1:].values
        similarity_matrix.iloc[i, j] = cosine_similarity(vec1, vec2)

In [ ]:
similarity_matrix

Name,CHEMBL1765744,CHEMBL1922143,CHEMBL1215171,CHEMBL1215524,CHEMBL1215304,Dasabuvir,Sofosbuvir,Daclatasvir,Velpatasvir
Name,,,,,,,,,
CHEMBL1765744,1.000000,1.000000,1.000000,1.000000,1.000000,0.852803,0.904534,0.738549,0.797724
CHEMBL1922143,1.000000,1.000000,1.000000,1.000000,1.000000,0.852803,0.904534,0.738549,0.797724
CHEMBL1215171,1.000000,1.000000,1.000000,1.000000,1.000000,0.852803,0.904534,0.738549,0.797724
CHEMBL1215524,1.000000,1.000000,1.000000,1.000000,1.000000,0.852803,0.904534,0.738549,0.797724
CHEMBL1215304,1.000000,1.000000,1.000000,1.000000,1.000000,0.852803,0.904534,0.738549,0.797724
Dasabuvir,0.852803,0.852803,0.852803,0.852803,0.852803,1.000000,0.942809,0.721688,0.668153
Sofosbuvir,0.904534,0.904534,0.904534,0.904534,0.904534,0.942809,1.000000,0.680414,0.755929
Daclatasvir,0.738549,0.738549,0.738549,0.738549,0.738549,0.721688,0.680414,1.000000,0.925820
Velpatasvir,0.797724,0.797724,0.797724,0.797724,0.797724,0.668153,0.755929,0.925820,1.000000
